In [ ]:
import pandas as pd
import pandas_utils as pu
# from meta_constants import MetaColumns
# from metadata_cleanup import create_unique_sample_id_column

In [ ]:
lib_id = "pluskal_nih"

In [ ]:
df1 = pd.read_csv("data/library/PNNL_GNPS_NIH_Library_Plate1_28.csv")
df2 = pd.read_csv("data/library/PNNL_GNPS_NIH_Library_Plate29_53.csv")
df = pd.concat([df1, df2], ignore_index=True)
df

In [ ]:
df["Platenumber"] = [int(plate_id.split("_")[1]) for plate_id in df["lib_plate_well"]]
df[["WellNumber", "WellLetter"]] = [[int(value[1:]),value[:1]] for value in df["well"]]
sorted_df = df.sort_values(by=["Platenumber", "WellLetter", "WellNumber"], ascending=[True, True, True])
sorted_df

In [ ]:
plate_dict = {
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 2,
    6: 2,
    7: 2,
    8: 2,
    9: 3,
    10: 3,
    11: 4,
    12: 4,
    13: 4,
    14: 4,
    15: 5,
    16: 5,
    17: 5,
    18: 5,
    19: 6,
    20: 6,
    21: 6,
    22: 6,
    23: 7,
    24: 7,
    25: 7,
    26: 7,
    27: 8,
    28: 8,
    29: 9,
    30: 9,
    31: 9,
    32: 9,
    33: 10,
    34: 10,
    35: 10,
    36: 10,
    37: 11,
    38: 12,
    39: 12,
    40: 12,
    41: 12,
    42: 13,
    43: 13,
    44: 13,
    45: 13,
    46: 14,
    47: 14,
    48: 14,
    49: 14,
    50: 15,
    51: 15,
    52: 15,
    53: 15

}

sorted_df["mixed_plate"] = [int(plate_dict.get(plate)) for plate in sorted_df['Platenumber']]

In [ ]:
sorted_df["mixed_location_plate"] = ["{:02d}P_{}{}".format(plate, letter, well) for plate, letter, well in zip(sorted_df["mixed_plate"], sorted_df["WellLetter"], sorted_df["WellNumber"])]

In [ ]:
sorted_df["lib_plate_well"] =  ["{}_{}".format(lib_id, plate_id) for plate_id in sorted_df["mixed_location_plate"]]

In [ ]:
sorted_df = sorted_df.sort_values(by=["mixed_plate", "WellLetter", "WellNumber"], ascending=[True, True, True])

In [ ]:
sorted_df

In [ ]:
sorted_df.to_csv("data/nih/nih_library.tsv", sep="\t", index=False)

# Reformatting unique sample id

In [ ]:
filename = r"C:\git\msn_library\data\library\mce_library_add_compounds_cleaned.tsv"
df = pu.read_dataframe(filename)
df

In [ ]:
df["well_location"] = [str(v).split("_")[1] for v in df["mixed_location_plate1"]]
df["well_location"]

In [ ]:
lib_id = "pluskal_mce"
plate_id_header="plate_id"
well_header="well_location"
df[MetaColumns.library_id] = lib_id
create_unique_sample_id_column(df, lib_id, plate_id_header, well_header)
df[[MetaColumns.library_id, MetaColumns.unique_sample_id]]

In [ ]:
pu.save_dataframe(df, filename)

# Changing filename

In [ ]:
import os
import pandas_utils as pu

path = r''
new_ending = '_id'
for filename in os.listdir(path):
    if filename.endswith('.raw') and not filename.endswith('_id.raw'):
        original_file = os.path.join(path, filename)
        newname = os.path.join(path, pu.add_filename_suffix(filename, "id"))
        print(newname)
        os.rename(original_file, newname)

In [ ]:
filename = r"C:\git\msn_library\data\library\mce_library_cleaned.tsv"
df = pd.read_csv(filename, sep="\t")
df

In [ ]:
df[df["CAS No."].duplicated(keep=False)]

# Add line break

In [ ]:
def remove_line_breaks(value):
    if isinstance(value, str):
        return value.replace("\n", " ")
    else:
        return value

df = df.apply(lambda v: remove_line_breaks(v))
df

In [ ]:
pu.save_dataframe(df, pu.add_filename_suffix(filename, "remove_line_break"))

# Get specific rows


In [ ]:
filename = r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned.tsv"
df = pd.read_csv(filename, sep="\t")
df

In [ ]:
df.loc[df["NCGC_ID"].isin(["NCGC00013053-07"])]

In [ ]:
old = r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned_old.tsv"
old_df = pd.read_csv(old, sep="\t")
old_df
# old_df.loc[old_df["NCGC_ID"].isin(["NCGC00013053-07"])]

In [ ]:
filtered_df = df.loc[df["unique_sample_id"].isin(["pluskal_mce_1D1_A20_id", "pluskal_mce_1D1_A21_id", "pluskal_mce_1D1_A22_id", "pluskal_mce_1D1_A23_id", "pluskal_mce_1D1_A24_id"])]

In [ ]:
pu.save_dataframe(filtered_df, pu.add_filename_suffix(filename, "A20_A24"))

In [ ]:
df2 = pu.read_dataframe(r"C:\git\msn_library\data\library\mce_library_cleaned.tsv")
df = pu.read_dataframe(r"C:\git\msn_library\data\library\mce_library_add_compounds.tsv")

In [ ]:
df = pu.read_dataframe(r"C:\git\msn_library\data\library\mce_library_all_cleaned.tsv")
df

In [ ]:
df["unique_sample_id_tims"] = df["plate_id"] + "_" + df["well_location"] + "_"

In [ ]:
df

In [ ]:
test = pu.read_dataframe(r"C:\git\msn_library\data\library\mce_library_all_cleaned_tims_id.tsv")

In [ ]:
test

In [ ]:
merge = pd.concat([df2, df])
merge

In [ ]:
pu.save_dataframe(merge, r"C:\git\msn_library\data\library\mce_library_all.tsv")